Testing multifits loading and using `gettp` on multiple files.

In [1]:
from dysh.fits.gbtfitsload import GBTFITSLoad

In [2]:
# Point it to the testdata directory on the root dysh directory.
testdata = "/home/sandboxes/psalas/Dysh/dysh/testdata"
# The folder with multiple SDFITS files.
data_path = f"{testdata}/AGBT18B_354_03.raw.vegas"

In [3]:
sdf = GBTFITSLoad(data_path, verbose=False)

In [4]:
sdf.summary()

,SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATIO
0,6.0,W49N,7.0,OffOn,1.0,45.077148,44.979549,4,2,4,1,128.090129,50.305025
1,7.0,W49N,7.0,OffOn,2.0,45.077148,44.979549,4,2,4,1,129.920889,50.435658


Up to this point, it works fine. It tells you there are multiple IFs (spectral windows).

Now, when we try to calibrate something, it breaks down.

In [5]:
ps_scans = sdf.getps(6, ifnum=0)

ValueError: Scans [6] not found in ONs or OFFs of bintable None

In [6]:
# Check what is going wrong by looking at `GBTFITSLoad.onoff_scan_list`

# These are the defaults.
plnum = 0
ifnum = 0
df = sdf.index(bintable=None, fitsindex=0)
# df = df[(df["PLNUM"] == plnum) & (df["IFNUM"] == ifnum)]
# dfon = sdf.select("_OBSTYPE", "PSWITCHON", df)
# dfoff = sdf.select("_OBSTYPE", "PSWITCHOFF", df)
# onscans = uniq(list(dfon["SCAN"]))  # wouldn't set() do this too?
# offscans = uniq(list(dfoff["SCAN"]))

In [7]:
df["IFNUM"]

0     2
1     2
2     2
3     2
4     2
5     2
6     2
7     2
8     2
9     2
10    2
11    2
12    2
13    2
14    2
15    2
16    2
17    2
18    2
19    2
20    2
21    2
22    2
23    2
24    2
25    2
26    2
27    2
28    2
29    2
30    2
31    2
Name: IFNUM, dtype: int16

The problem seems to be that while selecting the OffOn scans, it expects to find `ifnum=0`, but that is not part of the first file, so it crashes instead on moving on to the next file.